# Welcome to Exkaldi

In this section, we will build a decision tree. In order to train a triphone model, a decision tree is necessary.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Restorage lexicons generated in early step (3_prepare_lexicons).

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Then initialize a DecisionTree object. ___lexicons___ can be provided as a parameter.

In [ ]:
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons)

tree

___tree___ is an exkaldi __DecisionTree__ object.

Then prepare acoustic feature, hmm model and alignment.

In [ ]:
featFile = os.path.join(dataDir, "exp", "mfcc.ark")

feat = exkaldi.load_feat(featFile)

feat

Monophone HMM model and alignment have been generated in last step (5_train_mono_HMM-GMM). Now use them directly.

In [ ]:
hmmFile = os.path.join(dataDir, "exp", "train_mono", "final.mdl")

aliFile = os.path.join(dataDir, "exp", "train_mono", "final.ali")

As training the HMM model, we provide high-level API to train tree, but now we still introduce the training steps indetail.

### Train Dicision Tree in detail

#### 1. Accumulate statistics data

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_tree")

exkaldi.utils.make_dependent_dirs(outDir, False)

In [ ]:
treeStatsFile = os.path.join(outDir, "treeStats.acc")

tree.accumulate_stats(feat, hmmFile, aliFile, outFile=treeStatsFile)

#### 2. Cluster phones and compile questions.

In [ ]:
topoFile = os.path.join(dataDir, "exp", "topo")

questionsFile = os.path.join(outDir, "questions.qst")

tree.compile_questions(treeStatsFile, topoFile, outFile=questionsFile)

#### 3. Build tree.

We set 1000 as the target numbers of leaves of tree, that is more than pdfs of monophone model.

In [ ]:
model0 = exkaldi.hmm.load_hmm(hmmFile, hmmType="monophone", name="mono")

model0.info.gaussians

In [ ]:
targetLeaves = 1000

tree.build(treeStatsFile, questionsFile, targetLeaves, topoFile)

Decision has been built done. Look it.

In [ ]:
tree.info

Save the tree to file.

In [ ]:
treeFile = os.path.join(outDir, "tree")

tree.save(treeFile)

As mentioned above, we provided a high-level API to build tree directly.

### Train Dicision Tree in high-level API

In [ ]:
del tree
del model0

os.remove(treeStatsFile)
os.remove(questionsFile)
os.remove(treeFile)

In [ ]:
tree = exkaldi.hmm.DecisionTree(lexicons=lexicons)

tree.train(feat=feat, hmm=hmmFile, alignment=aliFile, topoFile=topoFile, numleaves=1000, tempDir=outDir)

Tree has been saved in directory automatically.